# 임금 예측목적 : 노동자들의 특성에 대한 선형 결합을 사용하여 노동자들의 임금을 예측하고 평균 제곱 오차 (MSE)와 조정된 MSE 와 r-squared 그리고 표본에서 벗어난 (out-of-sample) MSE 와 r-squared를 사용하여 예측 성능을 평가해보는 것임.데이터 출처 : U.S.Current Population Survey, 2012년 조사는 결혼하지 않은 여성 노동자들의 교육 수준 (고등학교 졸업, 대학교 재학, 대학교 졸업) 에 포커스 되어있으며, 표본 크기는 대략 4,000 정도이다.결과 변수 Y는 시간당 임금이며, X는 성별, 경험, 교육, 지리적 정보에 관련된 다양한 특성 지표들이다.# Dataset데이터셋은 다음 변수들을 포함한다.  1. wage : weekly wage  2. female : female dummy  3. cg : college Graduate Dummy  4. sc : some college dummy  5. hsg : High School graduate dummy  6. mw : mid-west dummy  7. so : south dummy  8. we : west dummy  9. ne : northeast dummy  10. exp1 : experience(year)  11. exp2 : experience squared (taken as experience squared/100)  12. exp3 : experience cubed (taken as experience cubed/1000)

## **Importing the necessary libraries and overview of the dataset**

In [ ]:
import numpy as npimport pandas as pdimport matplotlib.pyplot as pltimport seaborn as sns# to ignore warningsimport warningswarnings.filterwarnings("ignore")# pandas 지수표현 없애기pd.options.display.float_format = '{:.5f}'.format

### **Loading the data**

In [ ]:
df = pd.read_csv("data-wages_prediction/01_predicting_wages.csv")

In [ ]:
df.head()

,female,cg,sc,hsg,mw,so,we,ne,exp1,exp2,exp3,wage
0,0,0,0,1,0,0,0,1,33.00000,10.89000,35.93700,11.65909
1,0,1,0,0,0,0,0,1,27.00000,7.29000,19.68300,12.82500
2,0,0,1,0,0,0,0,1,13.00000,1.69000,2.19700,5.77703
3,0,1,0,0,0,0,0,1,2.00000,0.04000,0.00800,12.46875
4,1,1,0,0,0,0,0,1,15.00000,2.25000,3.37500,18.52500


### **데이터 정보 확인**

In [ ]:
df.info()

**Observations** :  * 3835개의 관찰 데이터값이 있고, 12개의 다른 변수들이 있음  * 데이터셋에 결측값이 없음  * 모든 더미 변수 (cg,sc,hsg, etc) 는 int 데이터 타입이고, 이항 변수로 0 과 1의 값을 가지고 있음

# 단변량 분석 (**Univariate Analysis**)## 데이터셋의 통계 요약 정보를 확인해보자.

In [ ]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
female,3835.00000,0.41799,0.49329,0.00000,0.00000,0.00000,1.00000,1.00000
cg,3835.00000,0.37627,0.48451,0.00000,0.00000,0.00000,1.00000,1.00000
sc,3835.00000,0.32386,0.46801,0.00000,0.00000,0.00000,1.00000,1.00000
hsg,3835.00000,0.29987,0.45826,0.00000,0.00000,0.00000,1.00000,1.00000
mw,3835.00000,0.28761,0.45271,0.00000,0.00000,0.00000,1.00000,1.00000
so,3835.00000,0.24355,0.42928,0.00000,0.00000,0.00000,0.00000,1.00000
we,3835.00000,0.21173,0.40859,0.00000,0.00000,0.00000,0.00000,1.00000
ne,3835.00000,0.25711,0.43710,0.00000,0.00000,0.00000,1.00000,1.00000
exp1,3835.00000,13.35319,8.63935,2.00000,6.00000,11.00000,19.50000,35.00000
exp2,3835.00000,2.52927,2.91055,0.04000,0.36000,1.21000,3.80250,12.25000


**Observations:**  * 평균 임금은 대략 시간당 15달러 이고, 최대값은 348달러 로 매우 높다.  * 42% 가 여성 노동자다.  * 평균 경력 년수는 13년이고, 최소 경력년수는 2년 최대 경력년수는 35년이다. 이는 다양한 경력 그룹으로 부터 추출된 다양화된 데이터임을 암시한다.  * 데이터의 38%가 대학 졸업자들이다.  * 32%가 대학 재학중 (some college)이고, 30% 가 고등학교 졸업생이다.  * 주의 주요 지리적 지역에 걸친 노동자들의 지리적 분포도 볼 수 있으며, 22-28% 사이에서는 거의 동일한 것으로 보이며, 이는 데이터가 여러 지역에서 균일한 방식으로 수집되고 있음을 다시 보여준다.

In [ ]:
df[['exp1','exp2','exp3','wage']].boxplot(figsize=(20,10))plt.show()

  * wage 변수에 대해서 아웃라이어가 있음을 확인할 수 있으며, 몇몇 사람들이 다른 사람들 보다 더 높은 수익을 얻는다는 점은 그럴사 하다.

# 이변량 분석 (**Bivariate Analysis**)

## 경력 년수와 임금간의 관계를 첫번째로 살펴보자

In [ ]:
sns.scatterplot(data = df, x = 'exp1',y = 'wage')plt.show()

  * 위에서 볼 수 있듯, 경험과 임금간의 상당한 관계까 없음을 알 수있다. 일반적인 가정을 해보면 높은 경력수준을 가진 사람들이 높은 임금을 받는 것은 여기서는 아닌 것처럼 보인다.

## dumpy 컬럼의 리스트를 만들어서 임금과의 관계를 확인해보자

In [ ]:
cols = df.select_dtypes('int').columns.to_list()cols

In [ ]:
for col in cols:    sns.kdeplot(data = df, x = 'wage', hue = df[col])    plt.show()

  * 이들 그림에서는 여성, 교육수준, 지리적 지역(주황색, 청색)의 임금 분포가 크게 다르지 않음을 볼 수 있다.

### Basic Model  * X 는 여성 여부(female indicator) (D) 와 다른 통제 변수 (W) 로 구성되며, 다른 통제 변수에는 constant, experience, experience squared, experience cubed, education indicators, and regional indicators 로 구성된다  * X는 p = 9 인 회귀 변수을 포함한다. (female, cg, sc, hsg, mw, so, we, ne, exp (9개 설명변수)

In [ ]:
####################  Linear and Quadratic specifications ##############################from sklearn.linear_model import LinearRegressionfrom sklearn import metricsY = df['wage'] # target variableX = df[['female', 'sc', 'cg', 'mw', 'so', 'we', 'exp1', 'exp2', 'exp3']] # regressors# defining the modelmodel = LinearRegression()# fit the Linear regression to the regressors and target variable.results = model.fit(X,Y) # train the modelprint("Intercept", results.intercept_) # beta_0

In [ ]:
# coeeficients of other regressorspd.DataFrame(results.coef_.reshape(1,-1), columns = X.columns)

,female,sc,cg,mw,so,we,exp1,exp2,exp3
0,-1.82640,2.48652,9.87081,-1.21424,0.40460,-0.25080,1.09646,-4.01339,0.46034


**Observations:**  * 대학 졸업자들이 더높은 임금을 받는것을 증명하든, cg 변수가 높은 계수를 가지고 있는것을 확인할 수 있습니다.  * Exp2 는 음의 상관계수를 가지는데, 이것은 exp2 일 수록 임금이 낮아지는 것을 의미합니다.  * female 요인의 계수는 음의 상관계수를 가지는데, 여성일수록 더 낮은 임금을 받는 다는 것을 의미합니다.

In [ ]:
# compute MSE and R^2y_pred = results.predict(X)print("R-squared", metrics.r2_score(Y, y_pred)) # R-squaredn = X.shape[0]k = X.shape[1]MSE_adj2 = ( n / (n - (k-1)) * np.mean(np.square(Y-y_pred)))print("MSE adj:", MSE_adj2)

  * r-sqaured가 매우 낮은 것으로 보아 모델 성능이 매우 좋지 않은 것으로 보인다.

In [ ]:
rsquared = metrics.r2_score(Y, y_pred)adj_rsquared = 1 - ((1- rsquared) * (n-1) / (n-k-1))print("Adj Rsquared:", adj_rsquared) # adjusted r-squared

## Flexible model  * X는 기본 모델에서 W의 모든 요소와 이들의 양방향 상호작용을 포함하는 W뿐만 아니라 D로도 구성된다.  * 양방향 상호작용이 의미하는 바는 두 변수들이 서로서로 배수이므로, 모델에서 2변수의 결합된 상호작용 효과를 포착할 수 있다.  * 상호작용 features를 만든 후에 X 는 p = 37 개의 회귀변수들을 포함한다.

In [ ]:
# Linear regression: Quadratic specificationfrom sklearn.preprocessing import PolynomialFeaturesX.drop('female', axis = 1, inplace = True)poly = PolynomialFeatures(interaction_only = True, include_bias = True)X_poly = poly.fit_transform(X) # creating polynomial features with degree = 2 (Quadratic)

In [ ]:
X_poly.shape

In [ ]:
X_poly = pd.DataFrame(X_poly, columns = poly.get_feature_names_out(X.columns))

In [ ]:
X_poly['female'] = df['female']X['female'] = df['female']# train the modelresults = model.fit(X_poly, Y)# beta_0print("Intercept", results.intercept_)

In [ ]:
pd.DataFrame(results.coef_.reshape(1,-1), columns = X_poly.columns).T.sort_values(by=0, ascending =False) # printing the co-efficients in a tabular format

,0
exp2,12.52185
cg,2.24049
cg so,1.70405
mw exp1,1.10761
mw exp3,0.90626
sc exp1,0.83908
we exp3,0.80500
cg exp1,0.78588
sc exp3,0.63303
we exp1,0.47192


**Observations:**  * cg*exp 의 계수는 꽤 높은편에 속하며, 경험 변수 배수 대학 학위 소지자의 임금이 높거나, 좋은 교육과 좋은 경험을 가진 사람들이 좋은 임금을 받는 다는것을 의미합니다.  * cg*mw 의 계수는 음수이며, 중서부의 대학 졸업생들이 좋은 페이를 받지 못한다는 것을 의미하고, 이것은 우리가 알고있는 기본 모델에서 대학 졸업생이 급여를 잘 받고 있다는 것과는 달리 중서부에 살고 있는 사람들에게는 좋지 않은 신호가 된다.  * sc * cg, so * we, mw * we, mw * so 의 계수는 거의 0이며 모델에 어떤 기여를 하지 않는 것을 보인다.

In [ ]:
# compute MSE 와 R^2 계산# compute MSE and R^2y_pred = results.predict(X_poly)print("R-squared", metrics.r2_score(Y, y_pred)) # R-squaredn = X_poly.shape[0]k = X_poly.shape[1]MSE_adj2 = ( n / (n - (k-1)) * np.mean(np.square(Y-y_pred)))print("MSE adj:", MSE_adj2)

  * 모델 성능이 기본 모델의 r-squared 와 비교했을 때 약간 향상했으나, 매우 적은양입니다.  * 전체 설명변수는 33개를 고려했고, cgsc, sowe, mwwe, mwso 는 거의 0입니다. 이는 모델에 거의 어떤 기여도 하지 않는 것을 보입니다.

In [ ]:
rsquared = metrics.r2_score(Y, y_pred)adj_rsquared = 1 - ((1- rsquared) * (n-1) / (n-k-1))print("Adj Rsquared:", adj_rsquared) # adjusted r-squared

  * 우리는 basic, flexible 두가지 예측 모델을 고려해보았습니다.  * 베이직 모델에서 설명 변수는 여성 여부 D와 다른 설명변수, constant, experience, experience squared, experience cubed, education and regional indicators 의 W를 포함했습니다. 기본 모델은 9개의 설명변수를 가집니다 .

여기서 p/n이 상당히 작다는 점을 고려하면 표본 선형 회귀는 모집단 선형 회귀를 상당히 잘 근사화해야 한다.

In [ ]:
p = 9n = X.shape[0]p/n

| p | R-squared_sample | R-squared_adj | MSE_adj  ---|---|---|---|---  basic reg | 9 | 0.0954 | 0.093 | 165.680  flexi reg | 33 | 0.1039 | 0.096 | 165.118

basic 모델과 flexible 모델의 성능이 거의 동일하며, felxible 모델이 조금 더 낫다는 결론을 내릴 수 있다. (R2가 살짝 더 높고, MSE가 살짝 더 낮다)

### **Basic Model on splitted data**

In [ ]:
####################  Linear and Quadratic specifications with Sample Splitting ##############################from sklearn.model_selection import train_test_split #using this we split the data into train and test in python.#Split using the function train_test_split# X_train, X_test, Y_train, Y_test = train_test_split( X, Y, test_size = 0.5, random_state = 101) # random split#split without the function train_test_splitind = int(X.shape[0] * 0.5 ) + 1 # test size= 0.5X_train = X.iloc[:ind, :]X_test = X.iloc[ind:, :]Y_train = Y.iloc[:ind]Y_test = Y.iloc[ind:]results = model.fit(X_train, Y_train)print("intercept", results.intercept_)

In [ ]:
pd.DataFrame(results.coef_.reshape(1,-1), columns = X_poly.columns).T.sort_values(by=0, ascending =False) # printing the co-efficients in a tabular format

,sc,cg,mw,so,we,exp1,exp2,exp3,female
0,2.38791,10.00758,-0.93945,0.00000,-0.00000,1.24133,-4.76748,0.56617,-1.65474


In [ ]:
y_pred = results.predict(X_test)print("R-squared", metrics.r2_score(Y_test, y_pred)) # R-squaredprint("MSE", metrics.mean_squared_error(Y_test, y_pred)) # MSE

### **Flexible model on splitted data**

In [ ]:
####################  Linear and Quadratic specifications with Sample Splitting ##############################from sklearn.model_selection import train_test_split #using this we split the data into train and test in python.#Split using the function train_test_split# X_train, X_test, Y_train, Y_test = train_test_split( X, Y, test_size = 0.5, random_state = 101) # random split#split without the function train_test_splitind = int(X.shape[0] * 0.5 ) + 1 # test size= 0.5X_train = X.iloc[:ind, :]X_test = X.iloc[ind:, :]Y_train = Y.iloc[:ind]Y_test = Y.iloc[ind:]results = model.fit(X_train, Y_train)print("intercept", results.intercept_)

In [ ]:
pd.DataFrame(results.coef_.reshape(1,-1), columns = X_poly.columns).T.sort_values(by=0, ascending =False) # printing the co-efficients in a tabular format

,0
1,0.00000
sc,2.33398
cg,6.48286
mw,-6.64347
so,0.00000
we,0.00000
exp1,-0.91369
exp2,16.08340
exp3,-0.10404
sc cg,0.00000


In [ ]:
y_pred = results.predict(X_test)print("R-squared", metrics.r2_score(Y_test, y_pred)) # R-squaredprint("MSE", metrics.mean_squared_error(Y_test, y_pred)) # MSE

### **Conclusion and recommendations:**| p | R-squared_test | MSE_test  ---|---|---|---  basic reg | 9 | 0.1027 | 154.584  flexi reg | 33 | 0.1046 | 154.260

  * 여기서는 데이터를 2등분하여 임의로 분할한 결과를 보고하고 flexible rule이 약간 더 잘 작동하는 것을 확인했으며, 이러한 수치는 서로 다른 데이터 분할에 따라 다르기 때문에 여러 데이터 분할에 걸쳐 평균 결과를 얻을 수 있습니다.  * 대졸자와 직장 경험이 좋은 사람들은 학력이 낮고 직장 경험이 적은 사람들에 비해 좋은 임금을 받고 있다는 사실을 확인했습니다.  * 남부와 중서부 지역에 사는 사람들은 제대로 된 임금을 받지 못하고 있다는 사실을 확인했습니다.

* * *

# 성별 임금 차이## 같은 직무 관련 특성을 가진 남성과 여성 사이의 예측 임금에 어떤 차이가 있을까요?

In [ ]:
df.groupby(['female']).mean().T

female,0,1
cg,0.35484,0.40611
sc,0.30197,0.35434
hsg,0.34319,0.23955
mw,0.28495,0.29133
so,0.23522,0.25515
we,0.22133,0.19838
ne,0.25851,0.25515
exp1,13.58020,13.03712
exp2,2.58659,2.44945
exp3,5.96494,5.59930


**Observations**  * 먼저 학력이 고등학교, 일부 단과대학 또는 단과대학에 해당하는 미혼남성과 미혼여성의 하위표본에 대한 위와 같은 기술통계를 살펴보았다.  * 남성의 시간당 평균임금은 16달러이고, 여성의 시간당 평균임금은 15달러였다. 따라서 같은 직무 관련 특성을 통제 하지 않으면 차이는 1정도 난다.  * 만약 이 특성을 좀더 살펴본다면, 우리는 평균적으로 남자들이 더 많은 경험을 가지고 있지만, 여자들은 대학 학위나 약간의 대학 교육을 가지고 있을 가능성이 더 높다는 것을 안다.  * 두 남성과 여성의 지리학적 분포는 거의 동일하다.

### **Basic Model**

In [ ]:
####################  Linear and Quadratic specifications ##############################from sklearn.linear_model import LinearRegressionfrom sklearn import metricsY = df['wage'] # target variableX = df[['female', 'sc', 'cg', 'mw', 'so', 'we', 'exp1', 'exp2', 'exp3']] # regressors# defining the modelmodel = LinearRegression()# fit the Linear regression to the regressors and target variable.results = model.fit(X,Y) # train the modelprint("Intercept", results.intercept_) # beta_0

**Observations:**  * r-squared 값이 매우 낮은 것으로 보아 모델 성능이 매우 낮다.  * cg 졸업생들이 높은 계수를 가지고 있는 것으로 보아 대학 졸업생들이 높은 임금을 받는 것으로 보인다.  * exp2 는 음의 계수를 가지고 있는 것으로 보아 exp2가 높을 수록 임금이 낮아지는 것을 알 수 있다.  * 여성 여부에 대한 계수가 음수인 것으로 보아 여성일 수록 더 낮은 임금을 가진다는 것을 의미한다.

### **Flexible model**

In [ ]:
# import PolynomialFeatures library to create polynomial featuresfrom sklearn.preprocessing import PolynomialFeatures# create an object of PolynomialFeatures with only the interaction termspoly = PolynomialFeatures(interaction_only=True,include_bias = False)#Dropping constant and female as we dont want to create interaction features for themX.drop(['const','female'],axis = 1,inplace = True)print(X.shape)# transform the data to add the polynomial features tooX_poly = poly.fit_transform(X)# convert to dataframeX_poly = pd.DataFrame(X_poly,columns= poly.get_feature_names_out(X.columns))print(X_poly.shape)X_poly['female'] = df['female']X['female'] = df['female']X_poly = sm.add_constant(X_poly)#defining the modelmodel = sm.OLS(Y, X_poly) # Linear regression/OLS object#fitting the modelresults = model.fit() # train the model# summary of the modelprint(results.summary())

우리는 다음의 선형 회귀모델을 추정했습니다.Y = $β_1D$ + $β^r_2W$ + ε.  1. D는 여성 여부에 대한 변수입니다. ( 여성이면 1 그렇지 않으면 0) W는 통제변수입니다.  2. Basic Model : W는 education and regional indicators, experience, experience squared, and experience cubed 로 구성됩니다.  3. Flexible model : W는 베이직 모델의 통제 변수들과 양방향 상호작용 변수들로 구성됩니다.

| Estimate | Standard Error | Confidence Interval  ---|---|---|---  basic reg | -1.8264 | 0.425 | [-2.659 -0.994]  flexi reg | -1.8800 | 0.425 | [-2.713 -1.047      * 위 테이블에서는 basic model 과 flexible model 의 추정된 회귀 계수와 95% 신뢰구간과 표준 오차를 볼 수 있습니다.  * 기본 회귀 모형과 유연 회귀 모형에 대한 결과는 매우 밀접하게 일치하고 있습니다.  * 시간당 임금의 예상 성별 격차는 약 -2달러입니다.  * 95% 신뢰 구간은 -2.7 에서 -1 달러 범위이며, -2는 신뢰 구간의 영역 내에 놓여 있으므로 동일한 특성을 가진 남성과 여성의 시간당 임금 차이가 통계적으로나 경제적으로 모두 유의하다는 결론을 내릴 수 있습니다. ] |

# **Partialling Out 에 대한 설명: (Linear Specification - Basic Model)****"Partialling-out" 이란, 회귀 계수 β1 의 개념적 이해를 제공하는 중요한 도구 입니다. partialling-out 단계는 다음과 같습니다.**  1. 오직 W만 사용하여 Y를 예측합니다. 그리고 잔차를 확인합니다. 즉, W의 Y에 대한 의존성을 제거합니다.  2. 오직 W만 사용하여 D를 예측합니다. 그리고 잔차를 확인합니다. 즉, W의 D에 대한 의존성을 제거합니다.  3. 그런 다음 1단계와 2단계의 잔차를 모형화하면, 이것은 Y가 D에만 의존하는 방법을 알려줄 것입니다.

## Step 1: 오직 W만 사용하여 Y를 예측하고 잔차를 확인합니다. 즉, W의 Y에 대한 의존성을 제거합니다.

In [ ]:
# target variableY = df['wage']#regressors WW = df[['sc', 'cg', 'mw' , 'so' , 'we' , 'exp1' , 'exp2' , 'exp3']]W = sm.add_constant(W)# Linear regression/OLS objectmodel = sm.OLS(Y, W)# train the modelresults = model.fit()# get the residualst_Y = results.residt_Y

## Step 2: 오직 W만 사용하여 D를 예측하고 잔차를 확인합니다. 즉, W의 D에 대한 의존성을 제거합니다.

In [ ]:
# target regressor D, i.e. female in our predicitng wage exampleD = df['female']#regressors WW = df[['sc', 'cg', 'mw' , 'so' , 'we' , 'exp1' , 'exp2' , 'exp3']]W = sm.add_constant(W)# Linear regression/OLS objectmodel = sm.OLS(D, W)# train the modelresults = model.fit()# get the residualst_D = results.residt_D

## Step3: 1단계와 2단계의 잔차를 모형화하면, 이것은 Y가 D에만 의존하는 방법을 알려줄 것입니다.

In [ ]:
# Run OLS coefficient get coefficients and 95% confidence intervalsX = sm.add_constant(t_D)Y = t_Y# Linear regression/OLS objectmodel = sm.OLS(Y, X)# train the modelresults = model.fit()# print the summary of the trained modelprint(results.summary())

# **Partialling Out 에 대한 설명: (Linear Specification - Flexible Model)**

In [ ]:
# target variableY = df['wage']#regressors WW = df[['sc', 'cg', 'mw' , 'so' , 'we' , 'exp1' , 'exp2' , 'exp3']]# create an object of PolynomialFeatures with only the interaction termspoly = PolynomialFeatures(interaction_only=True,include_bias = False)# transform the data to add the polynomial features tooX_poly = poly.fit_transform(W)X_poly = sm.add_constant(X_poly)#defining the modelmodel = sm.OLS(Y, X_poly) # Linear regression/OLS object # train the modelresults = model.fit()# get the residualst_Y = results.residt_Y

In [ ]:
# target regressorD = df['female']#Regressors wW = df[['sc', 'cg', 'mw' , 'so' , 'we' , 'exp1' , 'exp2' , 'exp3']]# create an object of PolynomialFeatures with only the interaction termspoly = PolynomialFeatures(interaction_only=True,include_bias = False)X_poly = poly.fit_transform(W)X_poly = sm.add_constant(X_poly)#defining the modelmodel = sm.OLS(D, X_poly)# train the modelresults = model.fit()# get the residualst_D = results.residt_D

In [ ]:
# Run OLS coefficient get coefficients and 95% confidence intervalsX = sm.add_constant(t_D)Y = t_Y# Linear regression/OLS objectmodel = sm.OLS(Y, X)# train the modelresults = model.fit()# print the summary of the trained modelprint(results.summary())

| Estimate | Standard Error  ---|---|---  basic reg | -1.8264 | 0.425  flexi reg | -1.8800 | 0.425  basic reg with partialling out | -1.8264 | 0.424  flexi reg with partialling out | -1.8800 | 0.423

# 결론:  * 성별 임금 격차에 대해 알아보기 위해 지금까지 논의한 아이디어를 적용해 보았다.  * 성별 임금 격차는 부분적으로 노동 시장에서 여성에 대한 진정한 차별을 반영할 수 있다.  * 이는 또한 부분적으로 여성이 다소 적은 임금을 받는 직업(예를 들어, 학교 교사)에 종사하게 될 가능성이 더 높다는 소위 선택 효과를 반영할 수도 있다.

* * *